In [89]:
import pandas as pd
import nltk
import re 
from nltk.tokenize import RegexpTokenizer

In [90]:
# Load data
df_people = pd.read_csv('..\\datasets\\1. Original Dataset\\people.csv')

In [91]:
df_people.head()

,id,Age,Gender,Education Level,Job Title,Years of Experience
0,0,32.0,Male,Bachelor's,Software Engineer,5.0
1,1,28.0,Female,Master's,Data Analyst,3.0
2,2,45.0,Male,PhD,Senior Manager,15.0
3,3,36.0,Female,Bachelor's,Sales Associate,7.0
4,4,52.0,Male,Master's,Director,20.0


In [92]:
# Load stopwords
stopwords = nltk.corpus.stopwords.words('english') 

In [93]:
def process_job_title(job_title):    
    if pd.isna(job_title):
        return []  # desctiption empty
    
    # Cleaning  (if desctiption not null)
    # Remove special characters and convert to lower case.
    job_title = re.sub("[^a-zA-Z]", " ", str(job_title)).lower()

    # Tokenize the job title
    tokenizer = RegexpTokenizer(r'\w+')
    job_title = tokenizer.tokenize(job_title)
    
    # Eliminate las stopwords (shoudn't be any)
    job_title = [word for word in job_title if word not in stopwords]
    
    return job_title


In [94]:
df_people['job_title_descriptions'] = df_people['Job Title'].apply(process_job_title)

In [95]:
df_people.head()

,id,Age,Gender,Education Level,Job Title,Years of Experience,job_title_descriptions
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,"[software, engineer]"
1,1,28.0,Female,Master's,Data Analyst,3.0,"[data, analyst]"
2,2,45.0,Male,PhD,Senior Manager,15.0,"[senior, manager]"
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,"[sales, associate]"
4,4,52.0,Male,Master's,Director,20.0,[director]


---
# Categorical Variables

In [96]:
from collections import Counter

# Flatten the list of lists and count occurrences
word_counts = Counter(word for words in df_people['job_title_descriptions'] for word in words)

# Convert to DataFrame (optional)
word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Count']).sort_values(by='Count', ascending=False)

print(word_counts_df)

word_counts_df.to_csv("..\\datasets\\2. Processed Dataset\\job_titles.csv", index=False)

           Word  Count
4        senior    151
5       manager     96
30       junior     90
3       analyst     68
9     marketing     67
..          ...    ...
74   management      1
76    assurance      1
77      capital      1
78  advertising      1
79    architect      1

[81 rows x 2 columns]


For this specific case, considering time limitations and efficiency, I leveraged ChatGPT to classify seniority levels and job categories. Otherwise, I would have implemented a machine learning approach, such as logistic regression, decision trees, or transformer-based NLP models like BERT or RoBERTa, for more precise classification.

In [97]:
# Define keywords for classification
seniority_levels = {
    "Intern": ["intern"],
    "Junior": ["junior", "entry"],
    "Mid": ["associate", "specialist", "coordinator", "executive"],
    "Senior": ["senior", "lead", "principal"],
    "Manager": ["manager", "head"],
    "Director": ["director", "vp", "chief", "c-level"],
    "Executive": ["ceo", "cto", "cfo", "coo", "president"]
}

job_categories = {
    "Engineering": ["engineer", "developer", "software", "data", "architect"],
    "Marketing": ["marketing", "brand", "advertising", "social", "content"],
    "Sales": ["sales", "account", "business", "customer", "representative"],
    "Finance": ["finance", "accountant", "analyst", "financial"],
    "HR": ["hr", "recruiter", "human", "talent"],
    "Operations": ["operations", "supply", "logistics", "project"],
    "Support": ["support", "customer", "service", "helpdesk"],
    "Design": ["designer", "ux", "graphic"],
    "Writing": ["writer", "copywriter", "editor"],
}

# Function to classify a job title containing multiple words
def classify_job_title(words):
    seniority = "Other"
    category = "Other"
    
    # Check if any word matches a seniority level
    for word in words:
        for level, keywords in seniority_levels.items():
            if word in keywords:
                seniority = level
                break
    
    # Check if any word matches a job category
    for word in words:
        for cat, keywords in job_categories.items():
            if word in keywords:
                category = cat
                break
    
    return seniority, category

# Apply classification
df_people[['Seniority', 'Category']] = df_people['job_title_descriptions'].apply(lambda x: pd.Series(classify_job_title(x)))

# Display results
df_people

,id,Age,Gender,Education Level,Job Title,Years of Experience,job_title_descriptions,Seniority,Category
0,0,32.0,Male,Bachelor's,Software Engineer,5.0,"[software, engineer]",Other,Engineering
1,1,28.0,Female,Master's,Data Analyst,3.0,"[data, analyst]",Other,Finance
2,2,45.0,Male,PhD,Senior Manager,15.0,"[senior, manager]",Manager,Other
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,"[sales, associate]",Mid,Sales
4,4,52.0,Male,Master's,Director,20.0,[director],Director,Other
...,...,...,...,...,...,...,...,...,...
370,370,35.0,Female,Bachelor's,Senior Marketing Analyst,8.0,"[senior, marketing, analyst]",Senior,Finance
371,371,43.0,Male,Master's,Director of Operations,19.0,"[director, operations]",Director,Operations
372,372,29.0,Female,Bachelor's,Junior Project Manager,2.0,"[junior, project, manager]",Manager,Operations
373,373,34.0,Male,Bachelor's,Senior Operations Coordinator,7.0,"[senior, operations, coordinator]",Mid,Operations


In [98]:
df_people.to_csv("..\\datasets\\2. Processed Dataset\\people.csv", index=False)